In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import os
from splinter import Browser

import pandas as pd
import numpy as np

In [3]:
# Scrape Mars News

executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [4]:
url="https://mars.nasa.gov/news/"
browser.visit(url)
html=browser.html
soup=BeautifulSoup(html, "lxml")

In [4]:
# Hey! New article on 8/22.

article=soup.find("div", class_="list_text")
news_title=article.find("div", class_="content_title").text
news_paragraph=article.find("div", class_="article_teaser_body").text
print(news_title)
print(news_paragraph)

NASA-JPL Names 'Rolling Stones Rock' on Mars
NASA's Mars InSight mission honored one of the biggest bands of all time at Pasadena concert.


In [147]:
# Scrape Featured Image

featured_image_url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(featured_image_url)
html=browser.html
soup=BeautifulSoup(html, 'lxml')

In [146]:
image = soup.find("img", class_="thumb")["src"]
print(image)

/spaceimages/images/wallpaper/PIA23345-640x350.jpg


In [149]:
featured_image_url=featured_image_url[0:-35]
print(featured_image_url)

https://www.jpl.nasa.gov


In [150]:
featured_image_url=featured_image_url+image_strip
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23345-640x350.jpg


In [4]:
# Scrape Mars Weather

mars_weather_url="https://twitter.com/marswxreport?lang=en"
browser.visit(mars_weather_url)
mars_weather_html=browser.html
soup=BeautifulSoup(mars_weather_html, 'lxml')

In [5]:
weather_tweet=soup.find("div", class_="js-tweet-text-container").text
print(weather_tweet)


We won’t be hearing from @MarsCuriosity or @NASAInSight for the next 2 weeks during Mars solar conjunction. Read more about why Mars missions go silent every 2 years: https://www.wral.com/mars-spacecraft-go-quiet-during-solar-conjunction/18595551/ …pic.twitter.com/fWruE2v151



In [6]:
weather_tweet_strip=weather_tweet[9:-27]
mars_weather=weather_tweet_strip
print(mars_weather)

 be hearing from @MarsCuriosity or @NASAInSight for the next 2 weeks during Mars solar conjunction. Read more about why Mars missions go silent every 2 years: https://www.wral.com/mars-spacecraft-go-quiet-during-solar-conjunction/18595551/ …


In [36]:
# Scrape Mars Facts
# Clean up the dataframe

mars_facts_df=pd.read_html("https://space-facts.com/mars/")
mars_facts_df=mars_facts_df[1]
mars_facts_df=mars_facts_df.rename(columns={0: "Description", 1: "Value"})
mars_facts_df=mars_facts_df.set_index("Description")

In [40]:
# Get into html table format

mars_facts_df_html=mars_facts_df.to_html()
print(mars_facts_df_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [105]:
# Loop for Mars Hemisphere images

# Empty List
Mars_hemispheres_images_url=[]

Mars_hems_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

# Strip the useless information from the url
Default_url=Mars_hems_url[:29]

browser.visit(Mars_hems_url)
Mars_hems_html=browser.html
soup=BeautifulSoup(Mars_hems_html, 'lxml')

# First branch
Hem_url=soup.find_all("div", class_="item")


for hemispheres in Hem_url:
    
    # Get the headline
    Headline_url=hemispheres.find('h3').text
    Headline_url=Headline_url[:-9]
    
    # Link to the individual hemisphere page
    Pic_home_url=hemispheres.find("a")["href"]
    
    browser.visit(Default_url+Pic_home_url)
    
    Pic_home_html=browser.html
    
    soup=BeautifulSoup(Pic_home_html, 'lxml')
    
    # Path to image within the hemisphere page
    Pic_html=soup.find("img", class_="wide-image")["src"]
    
    # Add to the default url
    Destination_url=Default_url + Pic_html
    
    # Create dictionary
    Mars_hemispheres_images_url.append({  \
                "Title": Headline_url, "Image_Url": Destination_url
    }
    )
    

In [106]:
# Check to make sure the titles are correct and the urls work properly.

Mars_hemispheres_images_url

[{'Title': 'Cerberus Hemisphere',
  'Image_Url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'Title': 'Schiaparelli Hemisphere',
  'Image_Url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'Title': 'Syrtis Major Hemisphere',
  'Image_Url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'Title': 'Valles Marineris Hemisphere',
  'Image_Url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]